# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

 # Оглавление:
* [Шаг 1. Общая информация о входных данных](#1)
* [Шаг 2. Предобработка данных](#2)
    * [2.1 Обработка пропусков](#2.1)
    * [2.2 Замена типа данных](#2.2)
    * [2.3 Обработака дубликатов](#2.3)
    * [2.4 Лемматизация](#2.4)
    * [2.5 Категоризация данных](#2.5)
* [Шаг 3. Ответы на вопросы](#3)
* [Шаг 4. Общий вывод](#4)
* [5. Чек-лист готовности проекта](#5)

## Шаг 1. Общая информация о входных данных
<a class="anchor" id="1"></a>

Изучим данные, для этого импортируем библиотеку, прочитаем файл и выведем первые 10 строк таблицы.

In [112]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Наблюдаем следующие проблемы:
+ столбец 'days_employed' содержит отрицательные значения, очевидно из-за ошибки при вводе;
+ одинаковые данные в столбце 'education' содержат разные регистры;
+ данные в столбцах 'days_employed' и 'total_income' было бы удобнее привести к целым числам;
+ данные в столбце 'purpose' не приведены к ограниченному набору категорий, что затруднит их обработку;

Выведем общую информацию о таблице.

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию.

Таблица содержит 12 столбцов с различными типами данных.
Разберем подробнее, какие имеются столбцы, и какую информацию они содержат:

+ children — количество детей в семье
+ days_employed — общий трудовой стаж в днях
+ dob_years — возраст клиента в годах
+ education — уровень образования клиента
+ education_id — идентификатор уровня образования
+ family_status — семейное положение
+ family_status_id — идентификатор семейного положения
+ gender — пол клиента
+ income_type — тип занятости
+ debt — имел ли задолженность по возврату кредитов
+ total_income — ежемесячный доход
+ purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

**Вывод**

Каждая строка таблицы содержит данные о клиенте банка. Для нашего исследования прежде всего интересны столбцы 'children', 'family_status', 'debt', 'total_income', 'purpose'. 

Необходимо решить следующие проблемы:
+ обработать пропуски, попутно выяснив их причину;
+ обработать некорректно введенные данные в столбце 'days_employed';
+ изменить тип данных столбцов 'days_employed' и 'total_income';
+ привести данные в столбце 'education' к одному регистру;
+ лемматизировать данные в столбце 'purpose';

<a class="anchor" id="2"></a>
## Шаг 2. Предобработка данных 

<a class="anchor" id="2.1"></a>
### Обработка пропусков

Проверим данные на наличие пропусков.

In [114]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видим, примерно для 10% клиентов отсутствует информация о трудовом стаже и доходах. Это может быть связано как с отсутствием данных, так и с фактическим отсутствием трудового стажа и доходов. Чтобы это выяснить, рассмотрим эти случаи внимательнее.

In [115]:
df[df['days_employed'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Как видим по столбцу 'income_type' (тип занятости), пропуски связаны с отсутсвием данных, а не с фактическим отсутсвием стажа и дохода. Чтобы пропущенные значения не повлияли на результаты исследования, заменим их на медианные значения.

Как мы помним, в некоторых строках трудовой стаж записан как отрицательное число, очевидно из-за ошибки при вводе данных. Чтобы корректно посчитать медианы, переведём стаж в положительные числа.

In [116]:
df['days_employed'] = df['days_employed'].abs()
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Заменим пропущенные значения на медианы и проверим результат.

In [117]:
days_employed_median = df['days_employed'].median(skipna=True)
total_income_median = df['total_income'].median(skipna=True)
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ради интереса оценим медианный стаж и медианный доход клиентов.

In [118]:
print('Медианный трудовой стаж составляет {:.0f} лет.'.format(days_employed_median / 365))
print('Медианный ежемесячный доход составляет {:.0f} рублей.'.format(total_income_median))


Медианный трудовой стаж составляет 6 лет.
Медианный ежемесячный доход составляет 145018 рублей.


**Вывод**

На этапе предобработки данных обнаружились пропуски и некорректно введённые данные. Мы исправили часть данных и обработали пропуски таким образом, чтобы это не повлияло на результаты исследования.

<a class="anchor" id="2.2"></a>
### Замена типа данных

Для более удобного восприятия данных заменим вещественные числа в столбцах 'days_employed' (трудовой стаж в днях) и 'total_income' (ежемесячный доход) на целые. Применим метод astype(); он универсален и подходит для нашей задачи. Метод astype() округляет все значения в меньшую сторону, поэтому предварительно применим метод round() и округлим значения до целых чисел.

In [119]:
df['days_employed'] = df['days_employed'].round().astype('int')
df['total_income'] = df['total_income'].round().astype('int')

Выведем общую информацию о таблице, чтобы проверить типы данных.

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Выведем первые 5 строк таблицы, чтобы оценить изменения визуально.

In [121]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

После того, как мы изменили тип данных для двух столбцов, информация стала восприниматься гораздо удобнее, при этом изменеия не повлияют на результаты исследования.

<a class="anchor" id="2.3"></a>
### Обработка дубликатов

Для корректной обработки дубликатов в таблице приведем данные в столбце 'education' (образование) к одинаковому регистру и проверим результат.

In [122]:
df['education'] = df['education'].str.lower()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Установим наличие дубликатов в таблице. В данных отсутствует уникальный идентификационный номер клиента, но полное совпадение данных настолько маловероятно, что в случае нахождения дубликата можно утверждать, что данные дублированы из-за ошибки при вводе. Используем метод duplicated(), он находит полностью идентичные строки, это как раз то, что нам требуется.

In [123]:
df.duplicated().sum()

71

Удалим все дубликаты из таблицы и проверим результат. 

In [124]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

На этапе предобработки данных выявлено наличие дубликатов и установлена причина их появления. Их удаление позволит провести анализ данных более точно.

<a class="anchor" id="2.4"></a>
### Лемматизация

Данные в столбце 'purpose' (цель получения кредита) не имеют единого стандарта и, судя по всему, записывались со слов клиентов. Это значит, что одна и та же цель получения кредита может быть записана совершенно разными словами, а это затрудняет анализ данных.
Лемматизируем столбец 'purpose' и оценим изменения. Для этого напишем функцию, которая лемматизирует слова в строке, а затем соединяет их в новую строку без разделителей.

In [125]:
from pymystem3 import Mystem
m = Mystem()

def lemmatize_text(text):
    lemmas = m.lemmatize(text)
    return "".join(lemmas).strip()

df['purpose'] = df['purpose'].apply(lemmatize_text)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилье
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилье
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительный образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба


**Вывод**

Мы лемматизировали столбец 'purpose' (цель получения кредита), что существенно облегчит анализ данных в этом столбце.

<a class="anchor" id="2.5"></a>
### Категоризация данных

Категоризация - это объединение данных в группы по заданному критерию. Для нашего исследования в первую очередь интересные следующие столбцы:
+ children — количество детей в семье
+ family_status — семейное положение
+ debt — имел ли задолженность по возврату кредитов
+ total_income — ежемесячный доход
+ purpose — цель получения кредита

Первые 3 столбца содержат категориальные данные, т.е. данные с ограниченным числом уникальных значений. Оставшиеся два столбца нам необходимо категоризовать.

Разобъем данные о доходах клиентов на 4 группы, условно "низкий доход", "средний доход", "высокий доход" и "очень высокий доход". Для этого нам нужно определить границы этих групп. 
Квантиль - это значение, которое заданная случайная величина не превышает с фиксированной вероятностью. Простыми словами, это как раз граница, которая делит наблюдения на части. Например квантиль (0,5) это значение, которое делит наблюдения на две равные части, т.е. медиана. Для данных о доходах найдем квантили (0,25), (0,5) и (0,75) что примерно даст нам границы наших групп.

In [126]:
quant_025 = df['total_income'].quantile(.25)
quant_05 = df['total_income'].median()
quant_075 = df['total_income'].quantile(.75)

# пытаюсь разобраться, почему группы получаются неравными
# судя по всему, из-за наличия дубликатов в столбце 'total_income'
# пробовал проводить вычисления до округления данных в этом столбце - ситуация не меняется
# не придумал, как решить эту проблему(

#print(df[quant_025 >= df['total_income']]['children'].count())
#print(df[(quant_025 <= df['total_income']) & (quant_05 >= df['total_income'])]['children'].count())
#print(df[(quant_05 <= df['total_income']) & (quant_075 >= df['total_income'])]['children'].count())
#print(df[quant_075 <= df['total_income']]['children'].count())

#print(df['total_income'].duplicated().sum())

Зная границы групп, разобъем данные на категории. Напишем функцию, которая принимает на вход ежемесячный доход клиента и возвращает категорию дохода. Затем с помощью этой функции добавим столбец с категорией дохода в таблицу.

In [127]:
def income_group(income):
    if income <= quant_025:
        return 'низкий'
    if (income <= quant_05) & (income > quant_025):
        return 'средний'
    if (income <= quant_075) & (income > quant_05):
        return 'высокий'
    return 'очень высокий'
#print(income_group(196000))
df['income_group'] = df['total_income'].apply(income_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилье,очень высокий
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилье,высокий
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительный образование,очень высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба,высокий


Теперь категоризируем столбец 'purpose' (цель кредита). На данный момент данные в нем лемматизированны, но одни и те же категории описаны разными словами. Посмотрим, какие цели лидируют на данный момент.

In [128]:
df['purpose'].value_counts().head(30)

автомобиль                                972
свадьба                                   791
на проведение свадьба                     768
сыграть свадьба                           765
операция с недвижимость                   675
покупка коммерческий недвижимость         661
операция с жилье                          652
покупка жилье для сдача                   651
операция с коммерческий недвижимость      650
жилье                                     646
покупка жилье                             646
покупка жилье для семья                   638
строительство собственный недвижимость    635
недвижимость                              633
операция со свой недвижимость             627
строительство жилой недвижимость          624
покупка недвижимость                      621
покупка свой жилье                        620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
на покупка свой автомобиль        

Как видим, цели получения кредита можно разделить на следующие категории:
+ недвижимость
+ автомобиль
+ свадьба
+ образование

Заменим строки в столбце 'purpose' на название категории, при условии что строка содержит одно из ключевых слов.

In [129]:
n=0 #сделаем счетчик успешных замен, чтобы понять, все ли значения мы заменили

for i in range(len(df['purpose'])):
     
    if ('жилье' in df.loc[i,'purpose']) or ('недвижимость' in df.loc[i,'purpose']) or ('дом' in df.loc[i,'purpose']):
            df.loc[i,'purpose'] = 'недвижимость'
            n += 1
    if ('автомобиль' in df.loc[i,'purpose']) or ('машина' in df.loc[i,'purpose']):
            df.loc[i,'purpose'] = 'автомобиль'
            n += 1
    if ('свадьба' in df.loc[i,'purpose']) or ('женитьба' in df.loc[i,'purpose']):
            df.loc[i,'purpose'] = 'свадьба'
            n += 1
    if ('образование' in df.loc[i,'purpose']) or ('учеба' in df.loc[i,'purpose']):
            df.loc[i,'purpose'] = 'образование'
            n += 1

print('Заменено {:.1%} значений'.format(n / (len(df['purpose']))))            
df.head()


Заменено 100.0% значений


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,недвижимость,очень высокий
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,недвижимость,высокий
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,образование,очень высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,высокий


Наше исследование затрагивает семейное положение клиента. В таблице имеется столбец 'family_status_id', который содержит идентификаторы семейного положения. Идентификатор будет удобнее использовать при работе с данными, т.к. он представляет собой однозначное число, в то время как статус описывается символами и разделителями. Составим "словарь" для  идентификаторов семейного положения.

In [130]:
family_dict = df[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


**Вывод**

На этапе категоризации данных мы разделили данные о доходе клиентов на 4 условные группы, также мы определили 4 категории для целей кредита и присвоили соответствующие значения всем строкам столбца 'purpose' (цель кредита). Кроме этого, мы составили "словарь" для идентификаторов семейного положения. Вышеперечисленные действия позволяют нам перейти непосредственно к ответам на вопросы исследования.

<a class="anchor" id="3"></a>
## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Визуально оценим статистику по количеству детей у клиентов.

In [131]:
df['children'].value_counts()


 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Очевидно, данные содержат ошибки. На данный момент в России зарегистрирована только одна семья с двадцатью детьми, а отрицательного количества детей быть не может. Заменим ошибочные данные на медианные.

In [132]:
children_true = df[(df['children'] != 20) & (df['children'] != -1)]['children'] # создадим столбец без ошибочных данных
children_median = children_true.median().astype('int') # посчитаем медиану
df['children'] = df['children'].replace([20, -1], children_median) #заменим значения на медианные
df['children'].value_counts()

0    14214
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Оценим зависимость между количеством детей и возвратом кредита в срок. Для этого в каждой категории посчитаем отношения клиентов, не вернувших кредит в срок, к общему количеству клиентов в этой категории.

In [133]:
child_0 = df[df['children'] == 0]['children'].count() # все клиенты без детей
debt_0 = df[(df['children'] == 0) & (df['debt'] == 1)]['children'].count() # клиенты без детей, просрочившие выплату кредита
conv_0 = debt_0 / child_0 # доля неплательщиков среди бездетных

child_1 = df[df['children'] == 1]['children'].count()
debt_1 = df[(df['children'] == 1) & (df['debt'] == 1)]['children'].count()
conv_1 = debt_1 / child_1

child_2 = df[df['children'] == 2]['children'].count()
debt_2 = df[(df['children'] == 2) & (df['debt'] == 1)]['children'].count()
conv_2 = debt_2 / child_2

child_345 = df[df['children'] >= 3]['children'].count() # для трех и более детей 
debt_345 = df[(df['children'] >= 3) & (df['debt'] == 1)]['children'].count()
conv_345 = debt_345 / child_345

print('Доля клиентов без детей, не вернувших кредит в срок составляет          {:.1%}'.format(conv_0))
print('Доля клиентов c 1 ребенком, не вернувших кредит в срок составляет       {:.1%}'.format(conv_1))
print('Доля клиентов с 2 детьми, не вернувших кредит в срок составляет         {:.1%}'.format(conv_2))
print('Доля клиентов с 3 и более детьми, не вернувших кредит в срок составляет {:.1%}'.format(conv_345))

Доля клиентов без детей, не вернувших кредит в срок составляет          7.5%
Доля клиентов c 1 ребенком, не вернувших кредит в срок составляет       9.2%
Доля клиентов с 2 детьми, не вернувших кредит в срок составляет         9.5%
Доля клиентов с 3 и более детьми, не вернувших кредит в срок составляет 8.2%


Доля клиентов, не вернувших кредит вовремя увеличивается с количеством детей, если рассматривать количество детей от 0 до 2. Клиенты с тремя и более детьми возвращают кредит вовремя чаще, чем клиенты с 1 и 2 детьми, но реже чем бездетные, хотя на результат для этой выборки могла повлиять ее относительно маленькая величина. Одно можно сказать точно: клиенты без детей чаще возвращают кредит вовремя.

**Вывод**

Клиенты без детей реже задерживают выплату кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Визуально оценим статистику семейного положения клиентов.

In [134]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Выведем "словарь" с идентификаторами семейного положения.

In [135]:
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Оценим зависимость между семейным положением и возвратом кредита в срок. Для этого в каждой категории посчитаем отношения клиентов, не вернувших кредит в срок, к общему количеству клиентов в этой категории.

In [136]:
fam_0 = df[df['family_status_id'] == 0]['family_status_id'].count() # женат / замужем
debt_fam_0 = df[(df['family_status_id'] == 0) & (df['debt'] == 1)]['family_status_id'].count()
conv_fam_0 = debt_fam_0 / fam_0

fam_1 = df[df['family_status_id'] == 1]['family_status_id'].count() # гражданский брак
debt_fam_1 = df[(df['family_status_id'] == 1) & (df['debt'] == 1)]['family_status_id'].count()
conv_fam_1 = debt_fam_1 / fam_1

fam_2 = df[df['family_status_id'] == 2]['family_status_id'].count() # вдовец / вдова
debt_fam_2 = df[(df['family_status_id'] == 2) & (df['debt'] == 1)]['family_status_id'].count()
conv_fam_2 = debt_fam_2 / fam_2

fam_3 = df[df['family_status_id'] == 3]['family_status_id'].count() # в разводе
debt_fam_3 = df[(df['family_status_id'] == 3) & (df['debt'] == 1)]['family_status_id'].count()
conv_fam_3 = debt_fam_3 / fam_3

fam_4 = df[df['family_status_id'] == 4]['family_status_id'].count() # Не женат / не замужем
debt_fam_4 = df[(df['family_status_id'] == 4) & (df['debt'] == 1)]['family_status_id'].count()
conv_fam_4 = debt_fam_4 / fam_4

print('Доля клиентов, не вернувших кредит вовремя:')
print()
print(family_dict.loc[0, 'family_status'], '       {:.1%}'.format(conv_fam_0))
print(family_dict.loc[1, 'family_status'], '      {:.1%}'.format(conv_fam_1))
print(family_dict.loc[2, 'family_status'], '        {:.1%}'.format(conv_fam_2))
print(family_dict.loc[3, 'family_status'], '             {:.1%}'.format(conv_fam_3))
print(family_dict.loc[4, 'family_status'], ' {:.1%}'.format(conv_fam_4))

Доля клиентов, не вернувших кредит вовремя:

женат / замужем        7.5%
гражданский брак       9.3%
вдовец / вдова         6.6%
в разводе              7.1%
Не женат / не замужем  9.8%


**Вывод**

Клиенты, не состоящие в браке, либо состоящие в гражданском браке, задерживают выплату кредита чаще клиентов c другим семейным положением.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Оценим зависимость между уровнем дохода и возвратом кредита в срок. Для этого в каждой категории посчитаем отношения клиентов, не вернувших кредит в срок, к общему количеству клиентов в этой категории.

In [137]:
income_low = df[df['income_group'] == 'низкий']['income_group'].count()
debt_income_low = df[(df['income_group'] == 'низкий') & (df['debt'] == 1)]['income_group'].count()
conv_income_low = debt_income_low / income_low

income_mid = df[df['income_group'] == 'средний']['income_group'].count()
debt_income_mid = df[(df['income_group'] == 'средний') & (df['debt'] == 1)]['income_group'].count()
conv_income_mid = debt_income_mid / income_mid

income_high = df[df['income_group'] == 'высокий']['income_group'].count()
debt_income_high = df[(df['income_group'] == 'высокий') & (df['debt'] == 1)]['income_group'].count()
conv_income_high = debt_income_high / income_high

income_vhigh = df[df['income_group'] == 'очень высокий']['income_group'].count()
debt_income_vhigh = df[(df['income_group'] == 'очень высокий') & (df['debt'] == 1)]['income_group'].count()
conv_income_vhigh = debt_income_vhigh / income_vhigh

print('Доля клиентов с низким доходом, не вернувших кредит в срок, составляет        {:.1%}'.format(conv_income_low))
print('Доля клиентов cо средним доходом, не вернувших кредит в срок, составляет      {:.1%}'.format(conv_income_mid))
print('Доля клиентов с высоким доходом, не вернувших кредит в срок, составляет       {:.1%}'.format(conv_income_high))
print('Доля клиентов с очень высоким доходом, не вернувших кредит в срок, составляет {:.1%}'.format(conv_income_vhigh))

Доля клиентов с низким доходом, не вернувших кредит в срок, составляет        8.0%
Доля клиентов cо средним доходом, не вернувших кредит в срок, составляет      8.5%
Доля клиентов с высоким доходом, не вернувших кредит в срок, составляет       8.9%
Доля клиентов с очень высоким доходом, не вернувших кредит в срок, составляет 7.1%


**Вывод**

В целом, вероятность задержки выплаты растет с уровнем дохода клиента, за исключением клиентов с очень высоким доходом; вероятность задержки с их стороны ниже чем у клиентов из любой другой категории.

- Как разные цели кредита влияют на его возврат в срок?

Визуально оценим статистику по цели кредита.

In [138]:
df['purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

Оценим зависимость между целью кредита и его возвратом в срок. Для этого в каждой категории посчитаем отношения клиентов, не вернувших кредит в срок, к общему количеству клиентов в этой категории.

In [139]:
estate = df[df['purpose'] == 'недвижимость']['purpose'].count()
estate_debt = df[(df['purpose'] == 'недвижимость') & (df['debt'] == 1)]['purpose'].count()
conv_estate = estate_debt / estate

vehicle = df[df['purpose'] == 'автомобиль']['purpose'].count()
vehicle_debt = df[(df['purpose'] == 'автомобиль') & (df['debt'] == 1)]['purpose'].count()
conv_vehicle = vehicle_debt / vehicle

edu = df[df['purpose'] == 'образование']['purpose'].count()
edu_debt = df[(df['purpose'] == 'образование') & (df['debt'] == 1)]['purpose'].count()
conv_edu = edu_debt / edu

wed = df[df['purpose'] == 'свадьба']['purpose'].count()
wed_debt = df[(df['purpose'] == 'свадьба') & (df['debt'] == 1)]['purpose'].count()
conv_wed = wed_debt / wed

print('Доля клиентов, задержавших выплату, в зависимости от цели кредита')
print()
print('Недвижимость {:.1%}'.format(conv_estate))
print('Автомобиль   {:.1%}'.format(conv_vehicle))
print('Образование  {:.1%}'.format(conv_edu))
print('Свадьба      {:.1%}'.format(conv_wed))

Доля клиентов, задержавших выплату, в зависимости от цели кредита

Недвижимость 7.2%
Автомобиль   9.4%
Образование  9.2%
Свадьба      8.0%


**Вывод**

Клиенты, собирающиеся преобрести недвижимость, самые дисциплинированные, на втором месте идут клиенты, планирующе свадьбу.

<a class="anchor" id="4"></a>
## Шаг 4. Общий вывод

В рамках нашего исследования мы провели предобработку данных:

+ обработали пропуски;
+ изменили тип данных там, где это требовалось;
+ провели работу с дублированными данными;
+ лемматизировали данные;
+ разбили данные на категории, составили "словарь";

Были получены ответы на вопросы исследования:

1. Клиенты без детей реже задерживают выплату кредита.
2. Клиенты, не состоящие в браке, либо состоящие в гражданском браке, задерживают выплату кредита чаще клиентов с другим семейным положением.
3. Вероятность задержки выплаты растет с уровнем дохода клиента, за исключением клиентов с очень высоким доходом; вероятность задержки с их стороны ниже чем у клиентов из любой другой категории.
4. Клиенты, собирающиеся приобрести недвижимость или сыграть свадьбу задерживают выплату кредита реже других.

Для получения более точных результатов, рекомендуется исследовать зависимость своевременной выплаты кредита от возраста, образования и типа занятости.

<a class="anchor" id="5"></a>
## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.